### Using Twitter API to Identify Geo-Location Trends 

In [7]:
#load our necessary libraries
import os
import re
import tweepy
from tweepy import OAuthHandler
import numpy as np
import pandas as pd

# text treatement
import nltk
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

#Wordcloud
from wordcloud import WordCloud,  ImageColorGenerator
import matplotlib.pyplot as plt
from PIL import Image

# Graphs
import plotly.io as pio
pio.renderers.default='browser'
import plotly.express as px

# transformers
#from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, AutoModelForTokenClassification

In [51]:
#loading our twitter libraries and credentials
import configparser
import tweepy

#read config

config = configparser.ConfigParser()
config.read('config.ini')

api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']

access_token= config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret']



# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)



In [56]:
# calling the api
api = tweepy.API(auth, wait_on_rate_limit=True)

# WOEID of London
woeid = 44418

trends_result = api.get_place_trends(id=44418)[0]['trends']
trends = {}

for i in trends_result:
    trends[i['name']] = i['tweet_volume']
trends_names = ' '.join(list(trends.keys()))

In [66]:
trends

{'#hottestdayoftheyear': None,
 'Srebrenica': 49432,
 '#conservativeleadership': None,
 '#synod': None,
 'Erik': 61403,
 '#MondayMotivation': 104191,
 'St Andrews': None,
 'Lea Michele': 23240,
 'Dion': None,
 'Collins': 12381,
 'Fanny Brice': None,
 'Monty Norman': None,
 'Vodafone': None,
 'Jamie Wallis': None,
 'Mark Francois': None,
 'Funny Girl': 26551,
 'Jen Shah': None,
 'Glee': 12892,
 'Universal Credit': None,
 'ASLEF': None,
 'Arnold': 20990,
 'Sophia Myles': None,
 'Ubisoft': 24240,
 'Maguire': 45763,
 'Murtough': 10025,
 'Bernie Ecclestone': None,
 'Guido': None,
 'Ava White': None,
 'Hunter Biden': 385934,
 'Roundhay': None,
 'Hirst': None,
 'Saliba': 11400,
 'Tens': 19243,
 'Met Office': None,
 'Amex': None,
 'Andreas Pereira': None,
 'Northern Ireland': 12869,
 'Broadway': 16094,
 'Morgan Gibbs': None,
 'Jamaica': 11719,
 'Ten Hag': 71847,
 'Sancho': 14095,
 'Ozil': None,
 'Prime Day': 26947,
 'Owen Jones': None,
 'Aircon': None,
 'Doucoure': None,
 'Klopp': 18221,
 'Nic

In [92]:

tweets = api.search_tweets(q='suavniex', tweet_mode='extended')
one_tweet = tweets[-1]
one_tweet._json



{'created_at': 'Mon Jul 11 18:27:03 +0000 2022',
 'id': 1546561725230612482,
 'id_str': '1546561725230612482',
 'full_text': 'RT @sahouraxo: "NATO is not a defense alliance, it\'s a war machine. Ask the people of Afghanistan, Iraq or Libya."\n\nhttps://t.co/sk9wMJV1IN',
 'truncated': False,
 'display_text_range': [0, 139],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'sahouraxo',
    'name': 'Sarah Abdallah',
    'id': 3293406121,
    'id_str': '3293406121',
    'indices': [3, 13]}],
  'urls': [],
  'media': [{'id': 1546171671199072256,
    'id_str': '1546171671199072256',
    'indices': [116, 139],
    'media_url': 'http://pbs.twimg.com/ext_tw_video_thumb/1546171671199072256/pu/img/qI9vJnThW7NO-Y_w.jpg',
    'media_url_https': 'https://pbs.twimg.com/ext_tw_video_thumb/1546171671199072256/pu/img/qI9vJnThW7NO-Y_w.jpg',
    'url': 'https://t.co/sk9wMJV1IN',
    'display_url': 'pic.twitter.com/sk9wMJV1IN',
    'expanded_url': 'https://twitter.com/wall

In [134]:
variables_we_need = ['created_at', 'id', 'full_text', 'entities', 'user', 'coordinates', 'retweet_count', 'favorite_count', 'lang']

def get_all_tweets(count=100, q='', lang='', tweet_mode='extended'):
    results = []

    tweets = tweepy.Cursor(api.search_tweets, q=q, lang=lang, tweet_mode=tweet_mode).items(count)

    for tweet in tweets:
        d = {}
        for variable in variables_we_need:
            d[variable] = tweet._json[variable]
        results.append(d)

    return results